In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv("https://bit.ly/wine_csv_data")
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data,target,test_size=0.2,random_state=42)

## 랜덤 포레스트

In [4]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1,random_state=42)
scores = cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# => 훈련 세트에 과대적합

0.9973541965122431 0.8905151032797809


In [5]:
rf.fit(train_input,train_target)
print(rf.feature_importances_) # 특성 중요도

[0.23167441 0.50039841 0.26792718]


In [6]:
rf = RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)
rf.fit(train_input,train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라 트리

In [7]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores = cross_validate(et,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 교차 검증 점수

0.9974503966084433 0.8887848893166506


In [9]:
et.fit(train_input,train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input,train_target,return_train_score=True,n_jobs=-1) # 결정 트리 수:100개
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [12]:
gb = GradientBoostingClassifier(n_estimators=500,learning_rate=0.2,random_state=42) # 결정 트리 수:500개
scores = cross_validate(gb, train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [13]:
gb.fit(train_input,train_target)
print(gb.feature_importances_)
# => 랜덤포레스트보다 일부 특성(당도)에 집중

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 그레이디언트 부스팅

In [15]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input,train_target,return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [16]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input,train_target)
result = permutation_importance(hgb, train_input,train_target, n_repeats=10,random_state=42,n_jobs=-1) # 특성 중요도 계산
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [17]:
result = permutation_importance(hgb, test_input,test_target,n_repeats=10,random_state=42,n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [18]:
hgb.score(test_input,test_target)

0.8723076923076923

In [19]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist',random_state=42)
scores = cross_validate(xgb, train_input,train_target,return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [20]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
